# Script 7 - Calculate new Monte Carlo p-values

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stat
import warnings
warnings.simplefilter('ignore') # filter some warning messages

ebus = ['California','Humboldt','Iberian/Canary','Benguela']
subs = ['Equatorward','Central','Poleward']

# Annual percentage of SST footprint

In [ ]:
#read in regression results
gamdf = pd.read_csv('../data/EBUS_glm_FullModel_Trends_Modified-Subregions.csv', index_col = 0)

#read in sst footprint
df1 = pd.read_csv('../data/upwellingfootprint_annualsummary_nearshore_Modified-Subregions.csv')
df1 = df1.rename({'Random_name':'Region'}, axis = 1)
gamdf

In [ ]:
# monte carlo permrutation. 2-tail t test
plt.figure(figsize=(10,10))
pvtmp1=list()
gpv = list()
nnam = list()
i = 1
for eb in ebus:
    for rg in subs:
        sebus = eb+'-'+rg
        nnam.append(sebus)
        #print('\n',sebus)

        gtmp = gamdf[(gamdf['EBUS']==eb)&(gamdf['Subregion']==rg)]
        dft = df1[df1['Region']==sebus]

        x=dft['Year'].values
        y=dft['Percent'].values
        slope, intercept, r_value, p_value, std_err = stat.linregress(x,y)

        mu=np.mean(y)
        sig = np.std(y)
        n= len(x)
        m=10000
        slps = np.full((m),np.nan)
        for k in range(m):
            ny=np.random.normal(mu, sig, n) #classic monte carlo
            #ny2=np.random.permutation(y) #permutation monte carlo
            slopes, intercept, r_value, p_values, std_err = stat.linregress(x,ny)
            slps[k]=slopes

        plt.subplot(4,3,i)
        sns.histplot(slps, kde=True, bins=20)
        pperc = np.percentile(slps,[2.5,97.5])

        plt.axvline(x=pperc[0],c='b')
        plt.axvline(x=pperc[1],c='b')
        plt.axvline(x=slope,c='r')
        if slope<=0:
            conf = (np.sum(slps<=slope)+1)/(m+1)
        else:
            conf = (np.sum(slps>=slope)+1)/(m+1)
        pvtmp1.append(np.round(conf,3))
        plt.grid()
        plt.title(sebus)
        if conf<=0.025:
            print(sebus, np.round(gtmp['p.value'].values[0],3),np.round(conf,3),'*')
        else:
            print(sebus, np.round(gtmp['p.value'].values[0],3),np.round(conf,3))
        gpv.append(np.round(gtmp['p.value'].values[0],3))
       
        i += 1
        
plt.tight_layout()
plt.show()


In [ ]:
dfp = pd.DataFrame({'EBUS-sub':nnam,'Footprint p-model':gpv,'Footprint p-montec':pvtmp1})
print('Modified Subregion p-values:')
dfp


# Seasonal SST means

In [ ]:
#read in data on SST footprint
df1 = pd.read_csv('../data/upwellingfootprint_seasonal_nearshore_Modified-Subregions.csv')
df1 = df1.rename({'Season_month':'Month'}, axis = 1)
df1['time'] = pd.to_datetime(df1[['Year', 'Month']].assign(day=1))
df1[['EBUS', 'Subregion']] = df1['Random_name'].str.split('-', 1, expand=True)

#reat in data on seasonal trends
gtre = pd.read_csv('../data/seasonal_trends_percent_Modified-Subregions.csv')
gtre['location']=gtre['EBUS']+'-'+gtre['Subregion']
gtre.head(10)

In [ ]:
# prepare data for montecarlo
df2 = df1[['EBUS','Subregion','time','Month','Percent']]
df2['season']=np.nan
ses = ['winter','spring','summer','fall']
for eb in ebus:
    if (eb==ebus[1]) | (eb==ebus[3]):
        smon=[6,9,12,3]   
    else:
        smon=[12,3,6,9]
    j=0
    for i in smon:
        a = (df2['EBUS']==eb)&(df2['Month']==i)
        df2['season'][a]=ses[j]
        j+=1
df2['year']=pd.DatetimeIndex(df2['time']).year
df2['location']=df2['EBUS']+'-'+df2['Subregion']
df2

In [ ]:
# monte carlo permutations
pvmonte = np.full((12,4),np.nan)
pvgam= np.full((12,4),np.nan)
i=0 # reg
for eb in ebus:
    for rg in subs:
        sebus = eb+'-'+rg
        j=0
        for senam in ses:
            #print(sebus)
            dft = df2[(df2['location']==sebus)&(df2['season']==senam)]
            gdft = gtre[(gtre['location']==sebus)&(gtre['Season']==senam.capitalize())]

            x=dft['year'].values
            y=dft['Percent'].values
            slope, intercept, r_value, p_value, std_err = stat.linregress(x,y)

            mu=np.mean(y)
            sig = np.std(y)
            n = len(x)
            m=10000
            slps = np.full((m),np.nan)

            for k in range(m):
                ny=np.random.normal(mu, sig, n) #classic monte carlo
                #ny2=np.random.permutation(y) #permutation monte carlo
                slopes, intercept, r_value, p_values, std_err = stat.linregress(x,ny)
                slps[k]=slopes

            if slope<=0:
                conf = (np.sum(slps<=slope)+1)/(m+1)
            else:
                conf = (np.sum(slps>=slope)+1)/(m+1)
            if conf<=0.025:
                print(sebus, senam,np.round(gdft['p.value'].values,3),np.round(conf,3),'*')
            else:
                print(sebus, senam,np.round(gdft['p.value'].values,3),np.round(conf,3))
            pvmonte[i,j]=np.round(conf,3)
            pvgam[i,j]=np.round(gdft['p.value'].values,3)
            j += 1
        i+=1
        


In [ ]:
eb = 'Iberian/Canary'
rg = 'Central'
sebus = eb+'-'+rg
j=0
senam = 'fall'
#print(sebus)
dft = df2[(df2['location']==sebus)&(df2['season']==senam)]
gdft = gtre[(gtre['location']==sebus)&(gtre['Season']==senam.capitalize())]

x=dft['year'].values
y=dft['Percent'].values
slope, intercept, r_value, p_value, std_err = stat.linregress(x,y)

mu=np.mean(y)
sig = np.std(y)
n = len(x)
m=10000
slps = np.full((m),np.nan)
                
for k in range(m):
    ny=np.random.normal(mu, sig, n) #classic monte carlo
    #ny2=np.random.permutation(y) #permutation monte carlo
    slopes, intercept, r_value, p_values, std_err = stat.linregress(x,ny)
    slps[k]=slopes

if slope<=0:
    conf = (np.sum(slps<=slope)+1)/(m+1)
else:
    conf = (np.sum(slps>=slope)+1)/(m+1)
if conf<=0.025:
    print(sebus, senam,np.round(gdft['p.value'].values,3),np.round(conf,3),'*')
else:
    print(sebus, senam,np.round(gdft['p.value'].values,3),np.round(conf,3))

sns.histplot(slps, kde=True, bins=20)
pperc = np.percentile(slps,[2.5,97.5])

plt.axvline(x=pperc[0],c='b')
plt.axvline(x=pperc[1],c='b')
plt.axvline(x=slope,c='r')
if slope<=0:
    conf = (np.sum(slps<=slope)+1)/(m+1)
else:
    conf = (np.sum(slps>=slope)+1)/(m+1)
pvtmp1.append(np.round(conf,3))
plt.grid()
plt.title(sebus + senam)

In [ ]:
dfp['winter SST p-model']=pvgam[:,0]
dfp['spring SST p-model']=pvgam[:,1]
dfp['summer SST p-model']=pvgam[:,2]
dfp['fall SST p-model']=pvgam[:,3]
dfp

In [ ]:
dfp['winter SST p-montec']=pvmonte[:,0]
dfp['spring SST p-montec']=pvmonte[:,1]
dfp['summer SST p-montec']=pvmonte[:,2]
dfp['fall SST p-montec']=pvmonte[:,3]
dfp

In [ ]:
#save data
dfp.to_csv('../data/monte-carlo_pvalues_classic_Modified-Subregions.csv')